In [23]:
import torch
import torch.nn as nn
import torchvision.models.resnet as resnet

from datasets import GetRepsDataset
import os
from torchvision.io import read_image, ImageReadMode
# from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from tqdm.notebook import trange, tqdm

import numpy as np

In [2]:
model = nn.Sequential(*list(resnet.resnet18(pretrained=True).children())[:-1])

In [3]:
patch_path = '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/'
dirs = os.listdir(patch_path)
dir_patch_dict = {}
for pid in dirs:
    patches = [os.path.join(patch_path, pid, x) for x in os.listdir(os.path.join(patch_path, pid))]
    dir_patch_dict[pid] = patches

In [16]:
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
batch_size = 64

transform=transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ConvertImageDtype(torch.float),
    normalize,
        ])

dataset = GetRepsDataset(dir_patch_dict, transform)
rep_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)

In [5]:
model(rep_loader.dataset.__getitem__(1)[0].unsqueeze(0)).reshape(-1,1).detach().numpy().shape

(512, 1)

In [6]:
rep_loader.dataset.__getitem__(1)

(tensor([[[ 0.3994,  0.5878,  0.9474,  ...,  0.8447,  0.9817,  1.5125],
          [ 0.1083,  0.7591,  1.0502,  ...,  1.1529,  1.4269,  1.3070],
          [ 0.3481,  0.3481,  0.3652,  ...,  1.2899,  1.3755,  1.3413],
          ...,
          [ 1.2043,  1.3070,  1.4954,  ..., -0.9534, -0.7308, -0.7137],
          [ 1.7865,  1.7865,  1.7865,  ..., -0.9705, -1.3130, -1.0904],
          [ 2.0434,  2.0263,  1.7180,  ..., -0.3712, -0.6965, -0.4911]],
 
         [[-1.3179, -1.0553, -0.8452,  ..., -0.9503, -0.5826,  0.0476],
          [-1.4055, -0.8452, -0.4076,  ..., -0.4601, -0.0049, -0.1275],
          [-1.3179, -1.2829, -1.1429,  ..., -0.3200,  0.1877,  0.1352],
          ...,
          [ 0.4678,  0.2402,  0.8004,  ..., -1.7556, -1.6506, -1.6856],
          [ 1.5882,  1.5707,  1.5357,  ..., -1.7906, -1.7031, -1.7731],
          [ 2.0259,  1.9559,  1.4832,  ..., -1.3529, -1.7206, -1.6155]],
 
         [[-0.3055, -0.1312,  0.0256,  ...,  0.0953,  0.3916,  1.0365],
          [-0.4450,  0.1651,

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def check_cuda():
    if torch.cuda.is_available():       
        device = torch.device("cuda")
        print(f'There are {torch.cuda.device_count()} GPU(s) available.')
        print('Device name:', torch.cuda.get_device_name(0))
        return device
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
        return device
    
device=check_cuda()

There are 1 GPU(s) available.
Device name: Tesla K80


In [9]:
model = model.to(device)

In [ ]:
rep_list = []
model.eval()
for img, path in tqdm(rep_loader):
    
    img = img.to(device)
    
    reps = model(img)
    rep_list.append(reps.detach().detach().cpu().numpy().reshape(batch_size, -1))

In [44]:
rep_array = np.concatenate(rep_list[:-1], axis=0)
np.save('all_reps',rep_array)

In [34]:
np.save('reps',np.concatenate(rep_list))

In [47]:
path_list = []
for img, path in tqdm(rep_loader):
    
    path_list.append(path)

  0%|          | 0/11175 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
newrep_list = rep_list[:-1]

In [45]:
path

['/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_22_level1_27873-10368-28897-11392.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_290_level1_40162-21633-41186-22657.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_264_level1_39138-8320-40162-9344.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_277_level1_39138-25730-40162-26754.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_78_level1_29921-29826-30945-30850.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_201_level1_36066-8320-37090-9344.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_34_level1_28897-5248-29921-6272.png',
 '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/GTEX-1I1GV-3026/tile_97_level1_30945-21633-3197

In [49]:
orig_paths = [x for xs in dir_patch_dict.values() for x in xs]

In [66]:
len(orig_paths[:-47])

715136

In [63]:
rep_array.shape

(715136, 512)

In [69]:
import pickle

with open('paths.pickle', 'wb') as file:
    pickle.dump(orig_paths[:-47], file)